We are implementing the Levin et al. (2004) colorization algorithm. 

The algorithm operates in $YUV$ space. Where The $Y$ represents the Luminance and the $U$,$V$ are the color channels. In our problem we will receive as the input image only a BW image with the $Y$ channel, and we need to solve the chrominance UV based on a marked image.
Afterwards we will solve a sparse system of linear equations to propagate color from the user scribbles.

Our implementation will use the following libraries:
* `numpy`: Matrix operations.
* `cv2` (OpenCV): Image I/O and RGB $\leftrightarrow$ $YUV$ conversion.
* `scipy.sparse`: Handling the large, sparse affinity matrix.

In [2]:
import numpy as np
import cv2
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

print(f"Numpy: {np.__version__}")
print(f"OpenCV: {cv2.__version__}")

Numpy: 2.2.6
OpenCV: 4.12.0


In the setup phase the images will be loaded and prepared. For this we must first normalize the images by converting them to `float64` in the range `[0, 1]`. This is critical for accurate variance calculations in the weighting function. Then we will convert them to the correct color space $YUV$. The algorithm uses the $Y$ channel from the original image and the $U, V$ values from the scribbles at marked locations.

In [3]:
def load_and_prep(image_path, scribbles_path):
    img_gray_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
    img_scrib_bgr = cv2.imread(scribbles_path, cv2.IMREAD_COLOR)

    # Normalize
    img_gray = img_gray_bgr.astype(np.float64) / 255.0
    img_scrib = img_scrib_bgr.astype(np.float64) / 255.0

    # Convert to YUV
    img_gray_yuv = cv2.cvtColor(img_gray.astype(np.float32), cv2.COLOR_BGR2YUV)
    img_scrib_yuv = cv2.cvtColor(img_scrib.astype(np.float32), cv2.COLOR_BGR2YUV)

    return img_gray_yuv, img_scrib_yuv

YUV_original, YUV_marked = load_and_prep("example.bmp", "example_marked.bmp")

In [ ]:
def show_image(name, image):
    cv2.imshow(name, image)
    cv2.waitKey(0)

    # It is for removing/deleting created GUI window from screen and memory
    cv2.destroyAllWindows()

This is the core of the Levin et al. algorithm. We treat the image as a graph where every pixel is a node connected to its neighbors.

**The Objective:**
We want to minimize the difference between a pixel's color $U(r)$ and the weighted average of its neighbors $U(s)$:
$$J(U) = \sum_{r} \left( U(r) - \sum_{s \in N(r)} w_{rs} U(s) \right)^2$$

The weight $w_{rs}$ represents the similarity between two pixels. If their intensities $Y(r)$ and $Y(s)$ are similar, the weight is large (color propagates). If they are different (an edge), the weight is small.
$$w_{rs} \propto e^{-(Y(r) - Y(s))^2 / 2\sigma_r^2}$$

**The Linear System ($Ax=b$):**
Since the cost function is quadratic, we can solve it by setting the derivative to zero, resulting in a sparse linear system:
1.  **Constrained Pixels (Scribbles):** $U(r) = \text{user\_val}$.
2.  **Unconstrained Pixels:** $U(r) - \sum w_{rs} U(s) = 0$.

In [ ]:
def get_weights_and_solve(Y_channel, marked_U, marked_V, mask):
    """
    Y_channel: The intensity channel (H, W)
    marked_U, marked_V: The U and V channels from the scribbled image
    mask: Boolean matrix where True indicates a scribbled pixel
    """
    H, W = Y_channel.shape
    num_pixels = H * W
    
    # Identify Neighbors (3x3 Window)
    # We iterate over the 3x3 window around each pixel to calculate weights
    # dx, dy are offsets from the center pixel
    window_range = [-1, 0, 1]
    
    # Lists to build the sparse matrix (LIL format concept, but using arrays for speed)
    rows = []
    cols = []
    data = []
    
    # Pre-calculate pixel indices
    img_indices = np.arange(num_pixels).reshape(H, W)
    
    # To estimate sigma (variance) locally, we look at the window
    # Simple approx: use mean variance or small constant if homogeneous
    # Levin paper uses local variance. Here we use a global estimate for simplicity 
    # or a local window calculation. 
    # For this snippet, we'll use a simplified standard deviation logic.
    
    print("Building affinity matrix...")
    for dy in window_range:
        for dx in window_range:
            if dy == 0 and dx == 0:
                continue # Skip self
            
            # Shift indices to find neighbors
            # We identify valid neighbors (pixels that don't fall off the edge)
            r_idx_y, r_idx_x = np.where(img_indices >= 0) # All pixels
            # Neighbor coordinates
            s_idx_y = r_idx_y + dy
            s_idx_x = r_idx_x + dx
            
            # Filter valid bounds
            valid = (s_idx_y >= 0) & (s_idx_y < H) & (s_idx_x >= 0) & (s_idx_x < W)
            
            # Valid center pixels (r) and neighbor pixels (s)
            r_y, r_x = r_idx_y[valid], r_idx_x[valid]
            s_y, s_x = s_idx_y[valid], s_idx_x[valid]
            
            # Flat indices for matrix construction
            r_flat = img_indices[r_y, r_x]
            s_flat = img_indices[s_y, s_x]
            
            # Calculate Weights
            # w_rs = exp( - (Y(r) - Y(s))^2 / (2 * sigma^2) )
            Y_r = Y_channel[r_y, r_x]
            Y_s = Y_channel[s_y, s_x]
            
            diff_sq = (Y_r - Y_s) ** 2
            sigma = 0.02 # Variance constant (tunable)
            w_rs = np.exp(-diff_sq / (2 * sigma**2))
            
            # Store triplet
            rows.extend(r_flat)
            cols.extend(s_flat)
            data.extend(-w_rs) # Negative because usually (I - W)x = 0

    # Create the sparse matrix A
    # Shape: (N, N)
    A = sparse.coo_matrix((data, (rows, cols)), shape=(num_pixels, num_pixels)).tolil()
    
    # Normalize rows (so weights sum to 1) and add diagonal
    # The diagonal is 1.0 (representing U(r))
    # Currently A has -w_rs. We need to normalize these weights.
    
    # Sum of weights per row
    sum_weights = np.abs(A.sum(axis=1).A.flatten())
    # Avoid division by zero
    sum_weights[sum_weights == 0] = 1.0 
    
    # Normalize A: divide each row by its sum
    # Using dia_matrix for efficient row scaling
    D_inv = sparse.diags(1.0 / sum_weights)
    A = D_inv @ A
    
    # Set diagonal to 1.0
    A.setdiag(1.0)
    
    # Apply Constraints (Scribbles)
    # For marked pixels, the equation is U(r) * 1 = marked_val
    # We replace the entire row in A with [0...0 1 0...0]
    
    mask_flat = mask.flatten()
    constrained_indices = np.where(mask_flat)[0]
    
    # Zero out rows for constrained pixels to remove neighbor influence
    # This is a slow operation in LIL, usually done better by constructing A with constraints known
    # But for clarity:
    A[constrained_indices, :] = 0
    A[constrained_indices, constrained_indices] = 1.0
    
    # Construct b vectors
    b_u = np.zeros(num_pixels)
    b_v = np.zeros(num_pixels)
    
    # Set values for constrained pixels
    flat_U = marked_U.flatten()
    flat_V = marked_V.flatten()
    
    b_u[constrained_indices] = flat_U[constrained_indices]
    b_v[constrained_indices] = flat_V[constrained_indices]
    
    # 6. Solve
    print("Solving linear system (this may take a moment)...")
    A_csr = A.tocsr()
    final_u = spsolve(A_csr, b_u)
    final_v = spsolve(A_csr, b_v)
    
    return final_u.reshape(H, W), final_v.reshape(H, W)

# Run the solver
# Define mask: where does the scribble image differ from the BW image?
# We look for pixels where color exists (U or V is not 0 or 0.5 depending on range)
# In normalized YUV (0-1), gray is typically around 0.5 for U and V.
# Let's assume significant deviation from the original YUV means a scribble.
diff = np.abs(YUV_marked - YUV_original)
# Sum diff over U and V channels
is_scribbled = (diff[:,:,1] + diff[:,:,2]) > 0.01

print(f"Found {np.sum(is_scribbled)} constrained pixels.")

# Solve
solved_u, solved_v = get_weights_and_solve(YUV_original[:,:,0], YUV_marked[:,:,1], YUV_marked[:,:,2], is_scribbled)

Found 17405 constrained pixels.
Building affinity matrix...
